1. CSP로 이진분류
2. 머신러닝 기법 이용(로지스틱) 데이터 분류
3. 전처리된 데이터를 전결합층 fully connected 엡실러 층3개로 해서 해보기 V

가상환경 환경 문제때문에 tensorflow가 안 깔렸었음... 왜지...정확한 이유는 모름 만약에 노 모듈 이난리 나면 base 아나콘다로 바꿀 것

In [1]:
!pip install numpy moabb

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mne.decoding import CSP
from moabb.datasets import BNCI2014_008
from moabb.paradigms import P300
from tensorflow.keras.optimizers import Nadam

C:\Users\kimso\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\kimso\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\kimso\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [3]:
import tensorflow

In [4]:
!pip install tensorflow

In [5]:
dataset = BNCI2014_008()

In [6]:
paradigm = P300()

In [7]:
X, labels, meta = paradigm.get_data(dataset=dataset)

C:\Users\kimso\anaconda3\Lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\Lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\Lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\Lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {ep

In [8]:
print("X shape:", X.shape)  # (33600, 8, 257) -> (샘플 수, 채널 수, 시간 축)
print("Labels shape:", labels.shape)  # (33600,)

X shape: (33600, 8, 257)
Labels shape: (33600,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [10]:
print("X shape:", X_train.shape)  # (33600, 8, 257) -> (샘플 수, 채널 수, 시간 축)
print("Labels shape:", y_train.shape)  # (33600,)

X shape: (26880, 8, 257)
Labels shape: (26880,)


In [11]:
csp = CSP(n_components=50, reg=None, log=True, norm_trace=False)

# CSP 변환 수행 (3D 형태 유지)
X_train_csp = csp.fit_transform(X_train, y_train)
X_test_csp = csp.transform(X_test)

Computing rank from data with rank=None
    Using tolerance 1e+02 (2.2e-16 eps * 8 dim * 5.7e+16  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=NonTarget covariance using EMPIRICAL
Done.
Estimating class=Target covariance using EMPIRICAL
Done.


In [12]:
print("CSP 변환 후 X_train_csp shape:", X_train_csp.shape)  
print("CSP 변환 후 X_test_csp shape:", X_test_csp.shape)  

CSP 변환 후 X_train_csp shape: (26880, 8)
CSP 변환 후 X_test_csp shape: (6720, 8)


In [13]:
scaler = StandardScaler()
X_train_csp = scaler.fit_transform(X_train_csp)
X_test_csp = scaler.transform(X_test_csp)

In [14]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_csp.shape[1],)),  # 입력층
    BatchNormalization(),  # 배치 정규화
    Dropout(0.2),  # 과적합 방지용 Dropout

    Dense(64, activation='relu'),  # 첫 번째 은닉층 (엡실론층)
    BatchNormalization(),
    Dropout(0.2),

    Dense(32, activation='relu'),  # 두 번째 은닉층 (엡실론층)
    BatchNormalization(),
    Dropout(0.2),

    Dense(1, activation='sigmoid')  # 출력층 (이진 분류 → sigmoid 사용)
])

C:\Users\kimso\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
print(np.unique(y_train))  # y_train에 어떤 값들이 있는지 확인
print(np.unique(y_test))

['NonTarget' 'Target']
['NonTarget' 'Target']


In [16]:
y_train = np.where(y_train == 'Target', 1, 0).astype(np.int32)
y_test = np.where(y_test == 'Target', 1, 0).astype(np.int32)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
X_train_csp.shape

(26880, 8)

In [19]:
optimizer = Nadam(learning_rate=0.0005)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# EarlyStopping 적용
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [20]:
history = model.fit(X_train_csp, y_train, epochs=50, batch_size=32, validation_data=(X_test_csp, y_test))

Epoch 1/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.6586 - loss: 0.6661 - val_accuracy: 0.8366 - val_loss: 0.4484
Epoch 2/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8235 - loss: 0.4762 - val_accuracy: 0.8366 - val_loss: 0.4462
Epoch 3/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8333 - loss: 0.4665 - val_accuracy: 0.8366 - val_loss: 0.4437
Epoch 4/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8342 - loss: 0.4564 - val_accuracy: 0.8366 - val_loss: 0.4432
Epoch 5/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8305 - loss: 0.4574 - val_accuracy: 0.8366 - val_loss: 0.4437
Epoch 6/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8298 - loss: 0.4605 - val_accuracy: 0.8366 - val_loss: 0.4427
Epoch 7/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8325 - loss: 0.4550 - val_accuracy: 0.8366 - val_loss: 0.4431
Epoch 8/50
840/840 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8352 - loss: 0.4489 - val_accuracy: 0

In [21]:
y_pred_train_nn = (model.predict(X_train_csp) > 0.5).astype("int32")
y_pred_test_nn = (model.predict(X_test_csp) > 0.5).astype("int32")

840/840 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [22]:
train_accuracy = accuracy_score(y_train, y_pred_train_nn)
test_accuracy = accuracy_score(y_test, y_pred_test_nn)

# 출력
print(f"🔹 FCN Train 정확도: {train_accuracy:.4f}")
print(f"🔹 FCN Test 정확도: {test_accuracy:.4f}")


🔹 FCN Train 정확도: 0.8325
🔹 FCN Test 정확도: 0.8366
